<a href="https://colab.research.google.com/github/ahfat/llama2-hf-lc/blob/main/Llama2_hf_lc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>